In [1]:
import json
from pymongo import MongoClient
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm

In [ ]:
# === Configuración ===
# json_file_path = "arxiv-metadata-oai-snapshot.json"
archivos = [f"arxiv-part-{i:02d}.json" for i in range(11)]
batch_size = 200
documentos_totales = 0
# max_workers = 4  # Ajusta según tu CPU y MongoDB

In [ ]:
# === Conectar a MongoDB ===
client = MongoClient("mongodb://mongo1:30001,mongo2:30002,mongo3:30003/?replicaSet=my-replica-set&readPreference=primary&appname=MongoDB%20Compass&ssl=false")
db = client["arxiv_db"]
collection = db["articles"]
# Crear índices para mejorar el rendimiento
collection.create_index("id")
collection.create_index("categories")

In [4]:
# === Función para insertar lote ===
def insert_batch(batch):
    try:
        collection.insert_many(batch, ordered=False)
    except Exception as e:
        print("Error al insertar batch:", e)

In [5]:
# === Contar líneas para tqdm ===
# def count_lines(filepath):
#     with open(filepath, 'r', encoding='utf-8') as f:
#         return sum(1 for _ in f)

# total_lines = count_lines(json_file_path)

In [ ]:
# === Lectura + carga paralela ===


# for archivo in archivos:
#     print(f"\n📂 Cargando {archivo}...")
#     # Contar líneas para tqdm
#     total_lines = sum(1 for _ in open(archivo, 'r', encoding='utf-8'))

#     with open(archivo, 'r', encoding='utf-8') as f, tqdm(total=total_lines, desc=f"Cargando {archivo}") as pbar:
#         batch = []
#         for line in f:
#             try:
#                 record = json.loads(line)
#                 record["pdf_source"] = f"https://arxiv.org/pdf/{record['id']}"
#                 batch.append(record)
#                 if len(batch) >= batch_size:
#                     insert_batch(batch)
#                     batch = []
#             except json.JSONDecodeError:
#                 continue
#             pbar.update(1)
#         if batch:
#             insert_batch(batch)

# print("🎉 Todos los archivos fueron cargados.")

for archivo in archivos:
    print(f"\n📂 Cargando {archivo}...")
    doc_count = 0
    
    # Usar with para manejo automático de archivos
    with open(archivo, 'r', encoding='utf-8') as f:
        # Usar tqdm para barra de progreso
        for line in tqdm(f, desc=f"Procesando {archivo}"):
            try:
                record = json.loads(line)
                # Solo agregar pdf_source si no existe
                if 'pdf_source' not in record:
                    record["pdf_source"] = f"https://arxiv.org/pdf/{record['id']}"
                
                # Insertar documentos de uno en uno para batches pequeños
                try:
                    collection.insert_one(record)
                    doc_count += 1
                    documentos_totales += 1
                except Exception as e:
                    print(f"Error insertando documento: {str(e)[:200]}")
                    continue
                
                # Liberar memoria
                del record
                
            except json.JSONDecodeError as e:
                print(f"Error decodificando JSON: {str(e)[:200]}")
                continue
            
            # Mostrar progreso cada 10,000 documentos
            if doc_count % 10000 == 0:
                print(f"  🔹 {doc_count} documentos procesados en {archivo}...")
                client.admin.command('ping')  # Mantener la conexión viva

    print(f"✅ Archivo {archivo} cargado con {doc_count} documentos.")

print(f"\n🎉 Todos los archivos cargados. Total insertados: {documentos_totales}")



📂 Cargando arxiv-part-00.json...
  🔹 10000 documentos procesados en arxiv-part-00.json...
  🔹 20000 documentos procesados en arxiv-part-00.json...
  🔹 30000 documentos procesados en arxiv-part-00.json...
  🔹 40000 documentos procesados en arxiv-part-00.json...
  🔹 50000 documentos procesados en arxiv-part-00.json...
  🔹 60000 documentos procesados en arxiv-part-00.json...
  🔹 70000 documentos procesados en arxiv-part-00.json...
  🔹 80000 documentos procesados en arxiv-part-00.json...
  🔹 90000 documentos procesados en arxiv-part-00.json...
  🔹 100000 documentos procesados en arxiv-part-00.json...
  🔹 110000 documentos procesados en arxiv-part-00.json...
  🔹 120000 documentos procesados en arxiv-part-00.json...
  🔹 130000 documentos procesados en arxiv-part-00.json...
  🔹 140000 documentos procesados en arxiv-part-00.json...
  🔹 150000 documentos procesados en arxiv-part-00.json...
  🔹 160000 documentos procesados en arxiv-part-00.json...
  🔹 170000 documentos procesados en arxiv-part-

In [13]:
'''
from pymongo import MongoClient

client = MongoClient("mongodb://localhost:27018/")
db = client["arxiv_db"]
collection = db["articles"]

# ⚠️ Eliminar todos los documentos
collection.delete_many({})
print("🗑️ Todos los documentos eliminados.")
'''

🗑️ Todos los documentos eliminados.
